# **ETL -IMAGES**

In [1]:
# [Config] Librerías
import pandas as pd
import os
import nibabel as nib
import re
import nibabel as nib
import pandas as pd
from datetime import datetime
import subprocess
import numpy as np


In [2]:
# [Config] Rutas
base_dir  = r"C:\Users\Hp\MACHINE\MRI\DATA"
output_dir  = "../Datos/IMAGES"
dcm2niix_path = r"C:\Users\Hp\dcm2niix_win\dcm2niix.exe"

df_clinical = pd.read_csv("../Datos/Clinical/ADNIMERGE.csv", low_memory=False)

In [3]:
# Atributos clínicos considerados
atributes = [
    # Identificación
    "PTID","VISCODE","EXAMDATE","DX",
    # Demografía
    "AGE","PTGENDER","PTEDUCAT",
    # Genética
    "APOE4",
    # Cognitivas
    "CDRSB","MMSE","ADAS13","FAQ",
    "RAVLT_immediate","RAVLT_learning","RAVLT_forgetting",
    "DIGITSCOR","TRABSCOR",
    # MRI volumétricos
    "Ventricles","Hippocampus","WholeBrain","Entorhinal","Fusiform","MidTemp","ICV",
]
df_clinical = df_clinical[atributes]
df_clinical["EXAMDATE"] = pd.to_datetime(df_clinical["EXAMDATE"], errors="coerce")

````
# [Función] Convertir DICOM -> NifTI
for root, dirs, files in os.walk(base_dir):
    if any(f.lower().endswith('.dcm') for f in files):
        rel_path = os.path.relpath(root, base_dir)
        out_path = os.path.join(output_dir, rel_path)
        os.makedirs(out_path, exist_ok=True)
        cmd = [
            dcm2niix_path,
            "-z", "n",       # ❌ No comprimir (.nii en lugar de .nii.gz)
            "-b", "n",       # ❌ No crear JSON
            "-o", out_path,  # 📁 Carpeta de salida
            "-f", "%p_%s",   # 🧩 Nombre del archivo (protocolo_series)
            root             # 📂 Carpeta de entrada con los DICOM
        ]
        print(f"📦 Convirtiendo: {root}")
        subprocess.run(cmd, check=True)
print("✅ Conversión DICOM → NIfTI completada.")
````

In [4]:
# [] Guardar atributos de las imágenes
records = []

for root, dirs, files in os.walk(output_dir):
    for file in files:
        if file.endswith(".nii"):
            full_path = os.path.normpath(os.path.join(root, file))

            # 🧩 Extraer ID del sujeto
            match_id = re.search(r'(\d{3}_S_\d{4})', full_path)
            sujeto_id = match_id.group(1) if match_id else None

            # 🗓️ Extraer fecha del estudio (YYYY-MM-DD)
            match_fecha = re.search(r'(\d{4}-\d{2}-\d{2})', full_path)
            fecha = match_fecha.group(1) if match_fecha else None

            # Leer metadatos del NIfTI
            try:
                img = nib.load(full_path)
                data = img.get_fdata()

                header = img.header
                shape = img.shape
                voxel_size = header.get_zooms()
                voxel_volume = np.prod(voxel_size)
                total_volume = voxel_volume*np.prod(shape)
                datatype = str(header.get_data_dtype())
                mean_intensity = np.mean(data)
                std_intensity = np.std(data)
                orientation = nib.aff2axcodes(img.affine)
                units = header.get_xyzt_units()

                records.append([
                    sujeto_id, fecha, shape, voxel_size, datatype,
                    voxel_volume, total_volume, mean_intensity, std_intensity, orientation, units, file, full_path
                ])
            except Exception as e:
                print(f"⚠️ Error leyendo {file}: {e}")

df_images = pd.DataFrame(records, columns=[
    "sujeto_id", "fecha_imagen", "shape", "voxel_size", "datatype",
    "voxel_volume_mm3", "total_volume", "mean_intensity", "std_intensity", "orientation", "units", "archivo", "ruta"
])

# Convertir fecha a tipo datetime
df_images["fecha_imagen"] = pd.to_datetime(df_images["fecha_imagen"], errors="coerce")

print(f"En total {len(df_images)} imágenes fueron cargadas correctamente.")

En total 291 imágenes fueron cargadas correctamente.


In [5]:
# Sujetos con más de 7 vistas
visitas_clinicas = df_clinical.groupby("PTID")["VISCODE"].nunique()
imagenes_por_sujeto = df_images.groupby("sujeto_id")["fecha_imagen"].count()
imagenes_por_sujeto.sort_values(ascending=False).head()

sujeto_id
027_S_0307    12
023_S_0126     8
014_S_0169     8
032_S_0214     8
032_S_0187     8
Name: fecha_imagen, dtype: int64

In [6]:
# Merge imágenes con información del examen

df_images_sorted = df_images.sort_values(["sujeto_id", "fecha_imagen"])
df_clinical_sorted = df_clinical.sort_values(["PTID", "EXAMDATE"])
grupos_img = df_images_sorted.groupby("sujeto_id")
grupos_clin = df_clinical_sorted.groupby("PTID")
asignaciones = []

# Iterar por sujeto
for sujeto in df_images_sorted["sujeto_id"].unique():
    imgs = grupos_img.get_group(sujeto).reset_index(drop=True)
    clin = grupos_clin.get_group(sujeto).reset_index(drop=True)
    
    n = min(len(imgs), len(clin))
    
    imgs = imgs.iloc[:n].copy()
    clin = clin.iloc[:n].copy()
    imgs["VISCODE"] = clin["VISCODE"]
    
    print(f"\nSujeto: {sujeto}")
    print(pd.DataFrame({
        "VISCODE": imgs["VISCODE"],
        "fecha_imagen": imgs["fecha_imagen"],
        "EXAMDATE": clin["EXAMDATE"]
    }))

    asignaciones.append(imgs)

df = pd.concat(asignaciones, ignore_index=True)
df = df.merge(
    df_clinical,
    left_on=["sujeto_id", "VISCODE"],
    right_on=["PTID", "VISCODE"],
    how="left"  # Usamos left para conservar todas las imágenes
)


Sujeto: 007_S_0101
  VISCODE fecha_imagen   EXAMDATE
0      bl   2005-12-20 2006-01-04
1     m06   2006-07-07 2006-07-06
2     m12   2007-01-29 2007-01-29
3     m18   2007-07-11 2007-07-11
4     m24   2007-12-18 2007-12-12
5     m30   2008-12-23 2008-07-18
6     m36   2010-01-28 2008-12-23

Sujeto: 007_S_0128
  VISCODE fecha_imagen   EXAMDATE
0      bl   2006-01-16 2006-02-06
1     m06   2006-08-14 2006-08-14
2     m12   2007-02-15 2007-02-15
3     m18   2007-08-20 2007-08-20
4     m24   2008-02-07 2008-02-07
5     m30   2009-02-12 2008-08-20

Sujeto: 007_S_0249
  VISCODE fecha_imagen   EXAMDATE
0      bl   2006-03-02 2006-03-14
1     m06   2006-09-27 2006-09-26
2     m12   2007-04-04 2007-04-03
3     m18   2007-10-03 2007-10-03
4     m24   2008-04-01 2008-03-31
5     m30   2009-03-12 2008-10-01
6     m36   2010-04-15 2009-03-11

Sujeto: 013_S_0240
  VISCODE fecha_imagen   EXAMDATE
0      bl   2006-03-20 2006-04-05
1     m06   2007-04-12 2006-10-25
2     m12   2007-10-24 2007-04-02
3 

* No todas las fechas de las imágenes coinciden con el EXAMDATE

In [7]:
# Lista de VISCODE válidos hasta mes 36
visitas_validas = ["bl", "m06", "m12", "m18", "m24", "m30", "m36"]
df = df[df["VISCODE"].isin(visitas_validas)].copy()
print(f"Se tienen {df.shape[0]} imágenes con su registro.")

Se tienen 279 imágenes con su registro.


In [8]:
# Contar vistas por pacientes
tabla_visitas = (
    df
    .assign(valor="O")  # marcamos presencia
    .pivot_table(index="sujeto_id", columns="VISCODE", values="valor", aggfunc="first", fill_value="X")
    .sort_index(axis=1)
)
tabla_visitas
conteo_visitas = (tabla_visitas == "O").sum(axis=1)
resumen_visitas = conteo_visitas.value_counts().sort_index()
tabla_resumen = pd.DataFrame({
    "Vistas": resumen_visitas.index,
    "Pacientes": resumen_visitas.values
})
print("Resumen de número de visitas por paciente:")
tabla_resumen


Resumen de número de visitas por paciente:


,Vistas,Pacientes
0,3,6
1,4,7
2,5,10
3,6,13
4,7,15


In [9]:
df = df.drop(columns=['fecha_imagen', 'PTID'])

In [10]:
df.columns

Index(['sujeto_id', 'shape', 'voxel_size', 'datatype', 'voxel_volume_mm3',
       'total_volume', 'mean_intensity', 'std_intensity', 'orientation',
       'units', 'archivo', 'ruta', 'VISCODE', 'EXAMDATE', 'DX', 'AGE',
       'PTGENDER', 'PTEDUCAT', 'APOE4', 'CDRSB', 'MMSE', 'ADAS13', 'FAQ',
       'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting', 'DIGITSCOR',
       'TRABSCOR', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'ICV'],
      dtype='object')

In [11]:
df.to_csv("../Datos/Clinical/ADNI_Images.csv", index=False)
df.to_csv("ADNI_Images.csv", index=False)